# Implementação de um Repository em TypeScript

In [ ]:
import type { Database as DatabaseType } from 'better-sqlite3'
import type { OAuthAuthorizationRequest, Client, OAuthToken } from './core'

export type AuthorizationRequestRepository = ReturnType<typeof SqliteAuthorizationRequestRepository>

export const SqliteAuthorizationRequestRepository = ({ db }: { db: DatabaseType }) => {
    const createStmt = db.prepare<OAuthAuthorizationRequest, OAuthAuthorizationRequest>("INSERT INTO oauth_authorization_request (code, client_id, project_id, code_challenge, user_id) VALUES (:code, :client_id, :project_id, :code_challenge, :user_id) RETURNING *")

    const getStmt = db.prepare<Pick<OAuthAuthorizationRequest, 'code'>, OAuthAuthorizationRequest>("SELECT * FROM oauth_authorization_request WHERE code = :code")

    return {
        // biome-ignore lint/style/noNonNullAssertion: <explanation>
        create: (data: OAuthAuthorizationRequest) => createStmt.get(data)!,

        get: (query: {
            code: OAuthAuthorizationRequest['code'];
        }) => {
            return getStmt.get({ code: query.code })
        }      
    };
}

export type ClientRepository = ReturnType<typeof SqliteClientRepository>

export const SqliteClientRepository = ({ db }: { db: DatabaseType }) => {
    const createStmt = db.prepare<Client, Client>("INSERT INTO client (id, project_id, secret, name, callback_endpoint) VALUES (:id, :project_id, :secret, :name, :callback_endpoint) RETURNING *");
    const getStmt = db.prepare<{
        id: Client['id'],
    }, Client>("SELECT * FROM client WHERE id = :id");

    return {
        create: (data: Omit<Client, 'id'>) => {
            const id = crypto.randomUUID();
            const output = createStmt.get({
                ...data,
                id
            });
            return output;
        },
        get: (query: Pick<Client, 'id'>) => {
            return getStmt.get({
                id: query.id,
            });
        }
    };
}

export type OAuthTokenRepository = ReturnType<typeof SqliteOAuthTokenRepository>

export const SqliteOAuthTokenRepository = ({ db }: { db: DatabaseType }) => {
    const createStmt = db.prepare<OAuthToken, OAuthToken>("INSERT INTO oauth_token (access_token, client_id, project_id, user_id, expires_in) VALUES (:access_token, :client_id, :project_id, :user_id, :expires_in) RETURNING *");

    return {
        create: (data: OAuthToken) => {
            const output = createStmt.get({ ...data });
            return output;
        },
    };
}